In [ ]:
## This is  a chatbot that uses tools to answer questions.


In [8]:
from langchain_openai import ChatOpenAI
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tools
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver


from datetime import datetime
from typing import Annotated
from typing_extensions import TypedDict

from dotenv import load_dotenv
_ = load_dotenv()


ModuleNotFoundError: No module named 'langchain_openai'

In [ ]:
llm = ChatOpenAI(model="gpt-4o")


In [ ]:
tavily_search = TavilySearchResults(max_results=2)

@tool
def get_current_date():
    """Returns the current date and time. Use this tool first for any 'time-based' questions."""
    return f"The current date is: {datetime.now().strftime('%d %B %Y')}"



In [ ]:
tools = [tavily_search, get_current_date]
llm_with_tools= llm.bind_tools(tools)

In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
## Create chatbot here....

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

In [ ]:
## Have to have a graph first to use the chatbot node
graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools",tool_node)


### NOTE FOR MYSELF: This is a conditional edge Im making to do the actual decision whether to use the tools/functions or
## not. This in particular is a prebuilt conditional edge...
graph_builder.add_conditional_edges("chatbot", tools_condition)


## Now if the tools are not used,
## then Im just going to return to the chatbot and process the tool output...
graph_builder.add_edge("tools","chatbot")

## NOTE: You have to set the entry point to the graph cycle...

graph_builder.set_entry_point("chatbot")

In [ ]:
## I add human in the look checkpoint here...

memory = MemorySaver()

graph = graph_builder.compile(checkpointer=memory,
                              interupt_before=["tools"])  #<-- interupt before executing tools





In [ ]:
from IPython.display import display, Image

# Visualize the graph...

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass ## visualization needs more stuff...



In [ ]:
## run graph here...

from IPython.display import Markdown, display

def render_markedown(md_string):
    display(Markdown(md_string))

def process_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message,tuple):
            print(message)
        else:
            message.pretty_print()
    return message

def process_query(query, config=None):
    inputs = {"messages": [("user", query)]}
    message = process_stream(graph.stream(inputs, config, stream_mode="values"))
    render_markdown(f"## Answer:\n{message.content}")









In [ ]:
user_input = "What is the weather like in Tokyo?"
config = {"configurable": {"thread_id": "1"}}

## The "config" is the SECODN positional argument to stream() or invoke()

events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)

for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()



In [ ]:
snapshot = graph.get_state(config)
print(snapshot.next)



In [ ]:
### Edit Agent Actions if I don't like what they are doing...


last_message = snatshot.values["messages"][-1]
print("Original Tool Call:", last_message.tools_calls)
print("Original Tool Call Message ID", last_message.id)



In [ ]:
from langchain_core.messages import AIMessage

# Copy the exisintg tool call and modify the query...
new_tool_call = last_message.tool_calls[0].copy()
new_tool_call["args"]["query"] = "What was the population of Singpore in 1990, when it was just a 'backwater' country?"



## Now you have to update with new message here...
new_message = AIMessage(
 content=last_message.content,
 tool_calls= [new_tool_call],
 id=last_message.id  ## <<-- Important to keep the same ID for replacement message
)

## print the neww tool call message...

print("New Tool Call:", new_tool_call)
print("New Tool Call Message ID", new_message.id)

In [ ]:
## Update the "State" with the new message...

graph.update_state(config, {"messages": [new_message]})
print("Updated tool call in the graph")
graph.get_state(config).values["messages"][-1].tool_calls



In [ ]:
events = graph.stream(None, config, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()